In [43]:
import bids
import matplotlib.pyplot as plt
import pandas as pd
import os
import ants
import numpy as np
import IPython.display as display
from ghost.phantom import Caliber137
import pydicom as pyd
from glob import glob
import SimpleITK as sitk

In [2]:
projdir = '/home/em2876lj/Projects/QA/QA_paper/project'
subses = pd.read_csv(os.path.join(projdir, 'code', 'slurm', 'all_sessions.txt'), delimiter=' ', names=['Subject', 'Session'])
layout = bids.BIDSLayout(root=projdir+'/rawdata', derivatives=projdir+'/derivatives')

In [3]:
# Setup the data sheet with info
csv_path = f'{projdir}/derivatives/fisp_temperature.csv'

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
else:
    df = subses.copy()
    df['n_black'] = None

In [20]:
plt.style.use('dark_background')

for i,row in df.iterrows():
    
    sub = row.Subject
    ses = row.Session
    
    # Check for row in dataframe    
    if (row['n_black']) or row['n_black']==0.0:
        print("Already matched")

    else:
        fig, ax = plt.subplots(1,1,figsize=(12,8))

        try:
            fisp_img = ants.image_read(layout.get(scope='raw', subject=sub, session=ses, extension='.nii.gz', suffix='PDw')[0].path)
        except IndexError:
            print("No FISP image")

        nx,ny,nz = fisp_img.shape
        plot_slices = np.arange(-7,6) + nz//2
        nslices = len(plot_slices)
        slices = [np.rot90(fisp_img[:, :, i]) for i in plot_slices]

        slices_2D = np.concatenate((np.concatenate(slices[0:nslices//2+1], axis=1),np.concatenate(slices[nslices//2:], axis=1)), axis=0)

        ax.imshow(slices_2D, cmap='gray')
        ax.axis("off")
        ax.set_title(f"{sub} {ses}")
        plt.show()
        n_black = input("Number of black vials")

        if n_black == '':
            n_black = None
        elif n_black.isnumeric():
            n_black = float(n_black)
        else:
            raise TypeError("Need to be a number")
        
        df.at[i, 'n_black'] = n_black

        display.clear_output(wait=True)


Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already matched
Already 

In [22]:
df.to_csv(csv_path)

In [3]:
# Larmour frequency
df_temp = pd.read_csv('/home/em2876lj/Projects/QA/QA_paper/project/derivatives/fisp_temperature.csv')

In [53]:
def get_dicom_name(folder):
    files = os.listdir(folder)
    for f in files:
        if ('.DS_Store' not in f) and ('.flywheel.json' not in f):
            f = os.path.join(folder, f)

            reader = sitk.ImageSeriesReader()
            
            if os.path.isdir(f):
                files = os.listdir(f)
                dicom_names = reader.GetGDCMSeriesFileNames(f)
                reader.SetFileNames(dicom_names)

            elif os.path.isfile(f):
                dicom_names = reader.GetGDCMSeriesFileNames(folder)
                reader.SetFileNames(dicom_names)
            
            return dicom_names
        
def get_w0(folder):
    dcm_name = get_dicom_name(folder)
    dcm = pyd.dcmread(dcm_name[0])
    return dcm[0x5200,0x9229][0][0x0018, 0x9006][0][0x0018,0x9098].value

In [56]:
fw_path = '/home/em2876lj/Projects/QA/QA_paper/data/UNITY-QA/data/UNITY-QA/SUBJECTS'
df_temp['w0'] = None
for i,row in df_temp.iterrows():
    sub = row.Subject
    ses = row.Session
    ses_path = f'{fw_path}/137-{sub.split("P")[1]}/SESSIONS'

    real_sessions = os.listdir(ses_path)
    sessions = [x.replace(' ', '').replace('_', '').replace('-','').replace('.','') for x in real_sessions]

    real_folder = real_sessions[sessions.index(ses)]

    acq_path = f'{ses_path}/{real_folder}/ACQUISITIONS/'
    T2_path = glob(acq_path+'/*T2*AXI*')[0]
    w0 = get_w0(T2_path+'/FILES')
    df_temp.at[i,'w0'] = w0

In [58]:
df_temp.to_csv(f'{projdir}/derivatives/fisp_temperature_w0.csv', index=None)